In [ ]:
import numpy as np
import pandas as pd
import os
import sys
import glob
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook
import pickle

import pydicom
import cv2
from PIL import Image
from mask_functions import rle2mask

from torch.utils.data import DataLoader, Dataset

%matplotlib inline

624 images has multiple-masks, 26% of non-empty-mask images, 

In [ ]:
#train_path = 'data/raw/dicom-images-train/*/*/*.dcm'
#test_path = 'data/raw/dicom-images-test/*/*/*.dcm'
train_path = 'data/processed/train/*'
test_path = 'data/processed/test/*'

train_fname_list = glob.glob(train_path)
test_fname_list = glob.glob(test_path)
len(train_fname_list), len(test_fname_list)#(10712, 1377)

In [ ]:
## convert all dcm images into png images ##

# train folder
for idx, raw_fname in enumerate(tqdm_notebook(train_fname_list)):
    dataset = pydicom.dcmread(raw_fname)
    fname = raw_fname.split('/')[-1][:-4]
    new_fname = 'data/processed/train/%s.png'%fname
    cv2.imwrite(new_fname, dataset.pixel_array)
    #if idx>10:
    #    break
# test folder
for idx, raw_fname in enumerate(tqdm_notebook(test_fname_list)):
    dataset = pydicom.dcmread(raw_fname)
    fname = raw_fname.split('/')[-1][:-4]
    new_fname = 'data/processed/test/%s.png'%fname
    cv2.imwrite(new_fname, dataset.pixel_array)
    #if idx>10:
    #    break

In [ ]:
train_mask = pd.read_csv('data/raw/train-rle.csv')
print('percentage of images having mask: ', (train_mask[' EncodedPixels']!=' -1').mean())
print('%d images out of %d having mask'%((train_mask[' EncodedPixels']!=' -1').sum(), len(train_mask.index)))
train_mask.head(5)

In [ ]:
train_mask.ImageId.nunique(), len(train_mask.index)
#train_mask.loc[0, ' EncodedPixels'].strip()

In [ ]:
from dataset_unet import SIIMDataset, mask2data, prepare_trainset

In [ ]:
train_dl, val_dl = prepare_trainset(BATCH_SIZE=16, NUM_WORKERS=24, SEED=1234, IMG_SIZE=256, debug=True)

In [ ]:
for i, (image, mask) in enumerate(train_dl):
    #image = image.to(device=device, dtype=torch.float)
    #target has keys: 'boxes', 'labels', 'masks', 'image_id', 'area', 'iscrowd'
    #masks = target['masks'].to(device=device, dtype=torch.float)
    if i == 3:
        break

In [ ]:
image.size(), mask.size(), mask.max(), mask.min()

In [ ]:
## study what is pneumothorax
fig=plt.figure(figsize=(15, 20))
for idx in range(16):
    ax = fig.add_subplot(4, 4, idx+1)
    plt.imshow(image[idx, 0, :, :], cmap=plt.cm.bone)
    is_nonempty = (mask[idx, 0, :, :].float().sum()>0).long().item()
    plt.imshow(mask[idx, 0, :, :], alpha=0.3, cmap="Reds")
    if is_nonempty:
        plt.title('Has Mask')
    else:
        plt.title('Empty')

In [ ]:
idx = 10
plt.imshow(np.rot90(image[idx, 0, :, :].numpy(), 0), cmap=plt.cm.bone)
plt.imshow(mask[idx, 0, :, :], alpha=0.3, cmap="Reds")

In [ ]:
#downloaded from kaggler's mask --dont know the original source
files = glob.glob('data/processed/masks_kaggler/*.png')

In [ ]:
files[0]

In [ ]:
##
rows = 10
l = [4,5,6,7]

cnt = 0
for idx, (img, mask) in enumerate(val_dl):
    for j in range(16):#BATCH_SIZE
        if j in l:
            continue
        not_empty = mask[j][0].sum()>0
        if not_empty:
            cnt+=1
            fig, (ax0, ax1, ax2) = plt.subplots(ncols=3, figsize=(12, 4))
            ax0.imshow(img[j][0].numpy(), plt.cm.bone)
            ax1.imshow(mask[j][0], vmin=0, vmax=1, cmap="Reds")
            #compare with kaggler's mask
            img_id = val_dl.dataset.data[idx*16+j]['img_id']
            mask_kaggler = plt.imread('data/processed/masks_kaggler/%s.png'%img_id)
            ax2.imshow(mask_kaggler, vmin=0, vmax=1, cmap="Blues")
            if not_empty.item():
                ax1.set_title('MyMask(Has Mask)')
            else:
                ax1.set_title('MyMask(Empty)')
            ax2.set_title('KagglerMask')
        if cnt>rows:
            break

In [ ]:
def show_dcm_info(dataset, fname):
    print("Filename.........:", fname)
    print("Storage type.....:", dataset.SOPClassUID)
    pat_name = dataset.PatientName
    display_name = pat_name.family_name + ", " + pat_name.given_name
    print("Patient's name......:", display_name)
    print("Patient id..........:", dataset.PatientID)
    print("Patient's Age.......:", dataset.PatientAge)
    print("Patient's Sex.......:", dataset.PatientSex)
    print("Modality............:", dataset.Modality)
    print("Body Part Examined..:", dataset.BodyPartExamined)
    print("View Position.......:", dataset.ViewPosition)
    if 'PixelData' in dataset:
        rows = int(dataset.Rows)
        cols = int(dataset.Columns)
        print("Image size.......: {rows:d} x {cols:d}, {size:d} bytes".format(
            rows=rows, cols=cols, size=len(dataset.PixelData)))
        if 'PixelSpacing' in dataset:
            print("Pixel spacing....:", dataset.PixelSpacing)
    print()

In [ ]:
random_fname_list = np.random.choice(train_fname_list, 20)

for idx, fname in enumerate(random_fname_list):
    print('fname_idx================%d'%idx)
    dataset = pydicom.dcmread(fname)
    show_dcm_info(dataset, fname)
    #plot_pixel_array(dataset)